In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import os

import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

# Cargar datos

In [2]:
from load_store_utils import cargar_lote_datos

# C:\Users\plane\OneDrive\Escritorio\COMPUTING SCIENCE\TFM_transformer_agents\data\Anomaly_detection\processed_data\
datos_test = 'HVAC_test_processeded.csv'
ruta_datos_procesados = '../data/Anomaly_detection/processed_data/'

new_data = cargar_lote_datos(m_samples=2, name= datos_test, ruta_datos=ruta_datos_procesados)
new_data

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power,delta_h,KPI,T_wb_out,month,day,hour
Timestamp,,,,,,,,,,,,,,,,
2021-03-30 09:00:00,22.434999,22.855000,21.5,20.489999,13.4,45.340000,35.959999,58.00,13.0,5.166,14.417635,2.003910,3.346269,3,30,9
2021-04-08 02:00:00,19.959999,20.337499,20.5,20.470000,11.6,29.504999,17.670000,37.25,0.0,0.000,11.189831,1.326863,2.602869,4,8,2


# Cargamos modelos

In [3]:
from load_store_utils import cargar_modelo

ruta = './models/anomaly_detection/'        # ruta a los modelos

nombre_scl = 'scaler_trained.pickle'        # el escalador de datos

# los modelos 
nombre_if = 'IF_trained.pickle'             # isolation forest
nombre_ee = 'EE_trained.pickle'             # eliptic envelope
nombre_lof = 'LOF_trained.pickle'           # local outlier factor
nombre_ocsvm = 'OCSVM_trained.pickle'       # one class svm
nombre_gm = 'GM_trained.pickle'             # gaussian mixtures
nombre_knn = 'dbscan_knn.pickle'            # dbscan_knn

scaler = cargar_modelo(name= nombre_scl, ruta_modelo=ruta)
IF = cargar_modelo(name=nombre_if, ruta_modelo=ruta)
EE = cargar_modelo(name=nombre_ee, ruta_modelo=ruta)
LOF = cargar_modelo(name=nombre_lof, ruta_modelo=ruta)
OCSVM = cargar_modelo(name= nombre_ocsvm, ruta_modelo= ruta)
GM = cargar_modelo(name= nombre_gm, ruta_modelo= ruta)
KNN_dbscan = cargar_modelo(name= nombre_knn, ruta_modelo= ruta)

Modelo cargado con éxito desde './models/anomaly_detection/scaler_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/IF_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/EE_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/LOF_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/OCSVM_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/GM_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/dbscan_knn.pickle'


# Escalado de datos

In [4]:
X_test_scl = scaler.transform(new_data)
#X_test_scl[0]

In [5]:
X_test_scl_df= pd.DataFrame(X_test_scl, columns= scaler.feature_names_in_, index= new_data.index)
X_test_scl_df

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power,delta_h,KPI,T_wb_out,month,day,hour
Timestamp,,,,,,,,,,,,,,,,
2021-03-30 09:00:00,0.341368,1.271058,-1.835308,0.773095,0.13502,-0.097942,-0.286834,-0.859605,1.348107,1.332514,0.764776,0.117016,0.166629,-0.996183,1.607679,-0.359652
2021-04-08 02:00:00,-0.341541,0.110630,-3.661276,0.763109,-0.24826,-1.435547,-1.826979,-1.859179,-0.884262,-0.884357,0.293183,0.023759,-0.196643,-0.771695,-0.906613,-1.370496


In [6]:
def parse_gm_predictions(gm_predictions): 
    gm_labels = []
    for instance in gm_predictions: 
    #print(instance)
        if max(instance) <0.99: gm_labels.append(-1)
        else:                   gm_labels.append(1)
    return gm_labels

def parse_knn_predictions(knn_predictions): 
    knn_labels = []
    knn_predicciones = KNN_dbscan.predict(X_test_scl_df)
    for instance in knn_predicciones: 
        if instance == -1: knn_labels.append(-1)
        else:              knn_labels.append(1)
    return knn_labels

# Predicciones 

In [7]:
# isolation forest 

if_predictions = IF.predict(X_test_scl_df)
print(f'Las predicciones del modelo Isolation Forest: \n{if_predictions}\n')

ee_predictions = EE.predict(X_test_scl_df)
print(f'Las predicciones del modelo Eliptic Envelope: \n{ee_predictions}\n')

lof_predictions = LOF.predict(X_test_scl_df)
print(f'Las predicciones del modelo Local Outlier Factor: \n{lof_predictions}\n')

ocsvm_predictions = OCSVM.predict(X_test_scl_df)
print(f'Las predicciones del modelo One Class SVM: \n{ocsvm_predictions}\n')

gm_predictions = GM.predict_proba(X_test_scl_df).round(4)

gm_labels = parse_gm_predictions(gm_predictions=gm_predictions)

print(f'Las predicciones del modelo Gaussian Mixtures: \n{gm_labels}\n')

knn_predictions = KNN_dbscan.predict(X_test_scl_df)
knn_labels = parse_knn_predictions(knn_predictions=knn_predictions)

print(f'Las predicciones del modelo KNN-DBSCAN: \n{knn_labels}')

Las predicciones del modelo Isolation Forest: 
[1 1]

Las predicciones del modelo Eliptic Envelope: 
[ 1 -1]

Las predicciones del modelo Local Outlier Factor: 
[-1 -1]

Las predicciones del modelo One Class SVM: 
[-1 -1]

Las predicciones del modelo Gaussian Mixtures: 
[1, 1]

Las predicciones del modelo KNN-DBSCAN: 
[1, 1]


In [8]:
model_names = ['isolation_forest', 'Eliptic_Envelope', 'Local_Outlier_Factor', 'One_Class_SVM', 'Gaussian_Mixtures', 'KNN_DBSCAN']
models = [IF, EE, LOF, OCSVM, GM, KNN_dbscan]
dct_models = dict(zip(model_names, models))

models_predictions = {}
for name, model in dct_models.items():
    
    if name == 'Gaussian_Mixtures': 
        pred = model.predict_proba(X_test_scl_df)
        predictions = parse_gm_predictions(pred)
    elif name == 'KNN_DBSCAN':
        pred = model.predict(X_test_scl_df)   
        predictions = parse_knn_predictions(pred)
    else: 
        predictions = model.predict(X_test_scl_df)

    models_predictions[name] = predictions

model_pred_df = pd.DataFrame(models_predictions, index= X_test_scl_df.index)
model_pred_df

,isolation_forest,Eliptic_Envelope,Local_Outlier_Factor,One_Class_SVM,Gaussian_Mixtures,KNN_DBSCAN
Timestamp,,,,,,
2021-03-30 09:00:00,1,1,-1,-1,1,1
2021-04-08 02:00:00,1,-1,-1,-1,1,1


# Pasando la predicción a open ai

In [9]:
model_pred_texto = model_pred_df.to_markdown()
print(model_pred_texto)

| Timestamp           |   isolation_forest |   Eliptic_Envelope |   Local_Outlier_Factor |   One_Class_SVM |   Gaussian_Mixtures |   KNN_DBSCAN |
|:--------------------|-------------------:|-------------------:|-----------------------:|----------------:|--------------------:|-------------:|
| 2021-03-30 09:00:00 |                  1 |                  1 |                     -1 |              -1 |                   1 |            1 |
| 2021-04-08 02:00:00 |                  1 |                 -1 |                     -1 |              -1 |                   1 |            1 |


In [10]:
from calcular_tokens import num_tokens_from_string

num_tokens_from_string(model_pred_df.to_markdown())

143

In [11]:
new_data.columns

columnas_base = ['T_Supply', 'T_Return', 'SP_Return', 'T_Saturation', 'T_Outdoor',
                 'RH_Supply', 'RH_Return', 'RH_Outdoor', 'Energy', 'Power']

datos_base = new_data[columnas_base]
display(datos_base)

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power
Timestamp,,,,,,,,,,
2021-03-30 09:00:00,22.434999,22.855000,21.5,20.489999,13.4,45.340000,35.959999,58.00,13.0,5.166
2021-04-08 02:00:00,19.959999,20.337499,20.5,20.470000,11.6,29.504999,17.670000,37.25,0.0,0.000


In [12]:
datos_base_texto = datos_base.to_markdown()


print(datos_base_texto)

| Timestamp           |   T_Supply |   T_Return |   SP_Return |   T_Saturation |   T_Outdoor |   RH_Supply |   RH_Return |   RH_Outdoor |   Energy |   Power |
|:--------------------|-----------:|-----------:|------------:|---------------:|------------:|------------:|------------:|-------------:|---------:|--------:|
| 2021-03-30 09:00:00 |     22.435 |    22.855  |        21.5 |          20.49 |        13.4 |      45.34  |       35.96 |        58    |       13 |   5.166 |
| 2021-04-08 02:00:00 |     19.96  |    20.3375 |        20.5 |          20.47 |        11.6 |      29.505 |       17.67 |        37.25 |        0 |   0     |


In [13]:
descripcion_lectura = datos_base.describe().to_markdown()
print(descripcion_lectura)

|       |   T_Supply |   T_Return |   SP_Return |   T_Saturation |   T_Outdoor |   RH_Supply |   RH_Return |   RH_Outdoor |   Energy |   Power |
|:------|-----------:|-----------:|------------:|---------------:|------------:|------------:|------------:|-------------:|---------:|--------:|
| count |    2       |    2       |    2        |      2         |     2       |      2      |      2      |       2      |  2       | 2       |
| mean  |   21.1975  |   21.5962  |   21        |     20.48      |    12.5     |     37.4225 |     26.815  |      47.625  |  6.5     | 2.583   |
| std   |    1.75009 |    1.78014 |    0.707107 |      0.0141421 |     1.27279 |     11.197  |     12.933  |      14.6725 |  9.19239 | 3.65291 |
| min   |   19.96    |   20.3375  |   20.5      |     20.47      |    11.6     |     29.505  |     17.67   |      37.25   |  0       | 0       |
| 25%   |   20.5787  |   20.9669  |   20.75     |     20.475     |    12.05    |     33.4637 |     22.2425 |      42.4375 |  3.25 

In [14]:
num_tokens_from_string(descripcion_lectura)

649

In [15]:
num_tokens_from_string(datos_base.to_markdown())

230

In [16]:
from calcular_tokens import num_tokens_from_messages

max_tokens_respuesta = 1000
mi_consulta = 'hazme un breve informe de lo que ves en la tabla de lecura y dime si cuales de estas lecturas se pueden considerar valores anómalos'

mi_prompt = [
    {'role': 'system', 'content': f'Eres un asistente de ayuda para un diagnostico de fallos en un sistema energético que genera respuestas concisas. Por un lado, \
                                    generas un informe resumido y breve de las instancias  con el número de registros, el maximo y minimo, media, etc. Da las unidades \
                                    de las variables. NO DÉS INFORMACIÓN SOBRE LAS VARIABLES DE LECTURA El usuario ya lo sabe.  Por otro lado, haces un breve resumen de \
                                    que lecuras se consideran anomalas según la serie de algoritmos de detección de anomalias; 1 significa valor típico y -1 valor atípico. \
                                    Adecúa tu respuesta a un maximo de {max_tokens_respuesta-100} tokens o más breve'}, 
                                
    {'role': 'user', 'content': f'A partir de la siguiente lectura:\n {datos_base}\n, con descripción: \n {descripcion_lectura}\n y la siguiente \
                                  predicción: \n {model_pred_texto}\n responde a mi consulta: \n{mi_consulta}'},

    {'role': 'assistant', 'content': 'INFORME: \n generas aqui tu informe \n \
                                      PREDICCIÓN: \n comentas de manera concisa y breve que lecuras se consideran anomalas y cuales no por ´mayority voting´\
                                      COCLUSIÓN: \n haz una breve conclusión'}
]

num_tokens_from_messages(mi_prompt)

1300

In [17]:
from open_ai_utils import enviar_promt_chat_completions_mode
from open_ai_utils import simular_respuesta_generativa

respuesta = enviar_promt_chat_completions_mode(
            mensaje= mi_prompt, 
            modelo="gpt-4-1106-preview", 
            maximo_tokens=max_tokens_respuesta, 
            aleatoriedad=0.1, 
            probabilidad_acumulada=1 )

simular_respuesta_generativa(respuesta)

INFORME:
- Número de registros: 2
- T_Supply: Mínimo 19.96°C, Máximo 22.435°C, Media 21.1975°C
- T_Return: Mínimo 20.3375°C, Máximo 22.855°C, Media 21.5962°C
- SP_Return: Mínimo 20.5°C, Máximo 21.5°C, Media 21°C
- T_Saturation: Mínimo 20.47°C, Máximo 20.49°C, Media 20.48°C
- T_Outdoor: Mínimo 11.6°C, Máximo 13.4°C, Media 12.5°C
- RH_Supply: Mínimo 29.505%, Máximo 45.34%, Media 37.4225%
- RH_Return: Mínimo 17.67%, Máximo 35.96%, Media 26.815%
- RH_Outdoor: Mínimo 37.25%, Máximo 58%, Media 47.625%
- Energy: Mínimo 0 kWh, Máximo 13 kWh, Media 6.5 kWh
- Power: Mínimo 0 kW, Máximo 5.166 kW, Media 2.583 kW

PREDICCIÓN:
- La lectura del 2021-03-30 09:00:00 es considerada anómala por Local Outlier Factor y One Class SVM.
- La lectura del 2021-04-08 02:00:00 es considerada anómala por Eliptic Envelope, Local Outlier Factor y One Class SVM.

CONCLUSIÓN:
La lectura del 2021-04-08 02:00:00 se considera más consistentemente anómala según la mayoría de los algoritmos aplicados. La lectura del 2021-0